<a href="https://colab.research.google.com/github/Rusildur/Pipeline_module/blob/main/pipeline_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from math import *
import numpy as np
import pandas as pd 
import openpyxl as ox

In [ ]:
#Начальные данные, константы для расчетов
P_0 = 101000  #атмосферное давление ПА

#Функции для расчетов, обработки данных

##Функции для предварительных расчетов

###Объем трубопровода (volume_pipeline)


In [ ]:
def volume_pipeline(dl: float, diam: int, st: int) -> float:
    """
    Функция расчета объема трубопровода.

    Принимает длину в метрах, диаметр и толщину стенки в мм, возвращает объем в м3.
    :param dl: длина в м
    :param diam: диаметр трубопровода в мм
    :param st: толщина стенки в мм
    :return: объем трубопровода в м3
    """
    radius = (diam / 2 - st) / 1000  # находим радиус и переводим в метры
    V = pi * dl * radius ** 2
    return V

###Напор трубопровода (flow_pipe)

In [ ]:
def flow_pipe(P: int, S: float, ro:float) -> float:
  """
  Функция для расчета напора трубопровода.

  Принимает давление в МПА, площадь отверстия, плотность вещества.
  Возвращает напор кг/с.

  :param P: давление в МПа
  :param S: площадь отверстия в м2
  :param ro: плотность вещества т/м3
  :return: напор кг/с
  """
  P = P * 1000000 # МПа в Па
  U = sqrt( 2 * (P - P_0) / ro * 1000)
  M_0 = 0.6 * S * U * ro 
  return M_0

###Площадь пролива (Strait_area)

In [ ]:
def Strait_area(massa: float, ro: float,  koef_str: int)-> float:
  """"
  Функция расчета площади пролива на неограниченное пространство.

  Принимает массу в тоннах, плотность вещества т/м3 и коэффициент разлития.
  Возвращает площадь пролива м2

  :param massa: масса в тоннах
  :param ro: плотность в т/м3
  :param koef_str: коэффициент разлития (5-неорганизованная территория, 20-грунт, 150 бетон )
  :return: площадь пролива м2
  """
  V_mass = massa / ro
  S = V_mass * koef_str
  return S

###Площадь отверстий (hole_area)

In [ ]:
def hole_area(diam: int, st: int, type = 'малая'):
  """
  Функция для расчета площади отверстия. 

  Принимает диаметр трубопровода в мм, толщину стенки в мм, тип разрыва.
  Возвращает площадь отверствия в м2

  :param diam: диаметр трубопровода в мм
  :param st: толщина стенки в мм
  :param: type: тип отверстия ('малая', 'гильотина', 'свищ')
  :return: площадь отверстия в м2

  """
  if type == 'свищ':
    return 0.0001
  elif type == 'гильотина':
    radius = (diam/2 - st)/1000
    S = pi * radius ** 2
    res = S *  0.179   
    return res 
  elif type == 'малая':
    radius = (diam/2 - st)/1000
    S = pi * radius ** 2
    res = S * 0.0072
    return res 

###Масса в аварии для гильотины ПП (massa_pp_gap)

In [ ]:
def massa_pp_gap(intensity: float, density: float, mass_in_pipeline: float, time = 6):
  """
  Функция расчета количества опасных веществ при разрыве нефтепровода.Расчет по ПП.
  Возвращает массу в тоннах. 
  :param intensity: интенсивность перекачки м3/сут
  :param density: плотность т/м3
  :param mass_in_pipeline: количество вещества в участке т
  :param time: время по ПП в часах (по умолчанию 6 часов)

  Возвращает:
  масса в тоннах в аварии разрыва нефтепровода.
  """
  intensity_hour = intensity * density / 24   # интенсивность перекачки т/ч
  poured_mass = intensity_hour * 6 * 0.25     # 25% за 6 часов перекачки по ПП
  full_mass = mass_in_pipeline + poured_mass # масса в участке + 25% за 6 часов по ПП
  return full_mass 

###Масса в аварии для свища ПП (massa_pp_puncture)

In [ ]:
def massa_pp_puncture(intensity: float, density: float, time_inspection = 3):
  """
  Функция расчета количества опасных веществ при проколе нефтепровода.Расчет по ПП.
  Возвращает массу в тоннах. 
  :param intensity: интенсивность перекачки м3/сут
  :param density: плотность т/м3
  :param time_inspection: время обхода в сутках (по умолчанию 3 дня)

  Возвращает:
  масса в тоннах в аварии свища нефтепровода.
  """
  intensity_hour = intensity * density / 24   # интенсивность перекачки т/ч
  poured_mass = intensity_hour * (time_inspection * 24)     # количество опорожненного вещества за время обхода
  full_mass = poured_mass * 0.02    # 2% от объема прокачки по ПП
  return full_mass 

###!!!!!!!!!Интенсивность испарения

In [ ]:
def Evaporation_404():
    """"
    Функция для расчета интенсивность испарения для нефти кг/м2час????
    
    
    """
    M = 200     #  Молярная масса нефти по расчетной Антона
    P_n = press_antuan(20, - 25)
    W = 10 ** -6 * sqrt(M) * P_n
    return W 

###Давление насыщенных паров (test)

Давление насыщенных паров через температуры

In [ ]:
def press_antuan(t_r, t_vsp ):
  P_n = exp(6.908 + 0.0433 * (t_r - 0.924 * t_vsp + 2.055) ) / (1047 + 7.48 * t_vsp)
  return P_n

Давление насыщенных паров Антуан

In [ ]:
def press_temp(t_r):
  A = 15.6782
  B = 2154.9
  C = -34.42
  P_n = 10 ** (A - B / (t_r + C) )
  return P_n

###Частота иницирующего события (test)

In [ ]:
def frequencies_pipline(diam):
  result = {'полная': '-' ,'частичная': '-'  }
  if diam < 75:
    result['полная'] = df_freq['полная'][0]
    result['частичная'] = df_freq['частичная'][0]
  elif 75 <= diam <= 150:
    result['полная'] = df_freq['полная'][1]
    result['частичная'] = df_freq['частичная'][1]    
  elif diam > 150:
    result['полная'] = df_freq['полная'][2]
    result['частичная'] = df_freq['частичная'][2] 
  else:
    return 'ERROR'

  return result 



###!!!!Частота реализации сценариев

In [ ]:
def scenarios():
  pass


###!!!!Классификация режимов сгорания облака

In [ ]:
def classification_cloud():
  pass 

##Функции расчета зон поражения

###Взрыв топливно-воздушной смеси (ТВС)

In [ ]:
def TVS_404(M_t):
    """ Функция расчета взрыва ТВС
    принимает массу испарившихся веществ
    """
    result = {'100': '-','53': '-', '28': '-', '12': '-','5': '-','3': '-'}
    betta = 1.14   # коэффициент для нефти
    E = M_t * 44 * 10 ** 6 * betta      # Удельное энерговыделение
    u = 43 * M_t ** (1/6)          # дефлаграция скорости фронта пламени для 5 класса
    x = 0.1
    while True:
        Rx = x / ((E / P_0) ** (1/3))
        if Rx < 0.34:
            Px1 = (u ** 2 / 340 ** 2) * (6/7) * (0.83/0.34 - 0.14/0.34 ** 2)
        else:
            Px1 = (u ** 2 / 340 ** 2) * (6/7) * (0.83/Rx - 0.14/Rx ** 2)
        
        delta_P = Px1 * 101
        if round(delta_P,2) >= 100.0:
          result['100'] = round(x,2)
        elif round(delta_P,2) >= 53.0:
          result['53'] = round(x,2)
        elif round(delta_P,2) >= 28.0:
          result['28'] = round(x,2)
        elif round(delta_P,2) >= 12.0:
          result['12'] = round(x,2)   
        elif round(delta_P,2) >= 5.0:
          result['5'] = round(x,2)        
        elif round(delta_P,2) >= 3.0:
          result['3'] = round(x,2) 
          return result 
          break 
        elif delta_P < 1:
          return result 
          break
        x += 0.1

###Пожар-вспышка

In [ ]:
def fire_flash_404(m, ro_g = 10, C_nkpr = 1.1):
    """" Модуль расчета пожара вспышки.
    Принимает массу, плотность испарившейся нефти, Нижний концетрационный предел нефти"""
    ro_g = 10       # плотность ГГ нефти
    C_nkpr = 1.1        # нижний концентрационный предел распространения пламени нефти
    R_f = 1.2 * 7.8 * (m / (ro_g * C_nkpr))** 0.33
    return R_f

###Пожар пролива

In [ ]:
def StraitFire_404(S):
    """" Функция для расчета пожара пролива (!!! пока только нефть).
    В тестовом варианте принимает значение площади пролива S. 
    Следует дополнить следующими входными данными:
    - вещество (молярная масса, плотность насыщенных паров, плотность )
    - скорость ветра
    - учесть в функции разный расчет Ef для других видов веществ
    """
    result = {'10,5': '-', '7,0': '-', '4,2': '-', '1,4': '-'}
    d = sqrt(4 * S / pi)    #эффективный диаметр пролива
    m_u = 0.04          #удельная скорость выгорания топлива
    ro_parov = 218/22.4     #плотность насыщенных паров при испарении
    p_okr_pro = 1.2250      #плотность окружающего воздуха 
    w_0 = 0             #скорость ветра
    Ef = 140 + exp(-0.12 * d) + 20 * (1 - exp(-0.12 * d))       # среднеповерхностная интенсивность теплового излучения пламени
    theta = 0       # угол отклонения пламени от вертикали под действием ветра. (принимается равной 0, если не учитываем ветер)
    u_n = w_0 / np.cbrt(m_u * 9.81 * d / ro_parov)
    if u_n < 1:
        theta = 0
    else:
        theta = u_n ** (-0.5)
    theta = 0 
    
    if u_n >= 1:
        L = 55 * d * (m_u / (p_okr_pro * sqrt(9.81 * d)))** 0.67 * u_n ** 0.21
    else:
        L = 42 * d * (m_u / (p_okr_pro * sqrt(9.81 * d)))** 0.61
    x = 1
    while True:
        a = 2 * L / d
        b = 2 * x / d
        A = sqrt((a**2 + (b + 1)**2 - 2 * a * (b + 1) * sin(theta)))
        B = sqrt((a**2 + (b - 1)**2 - 2 * a * (b - 1) * sin(theta)))
        C = sqrt(1 + (b**2 - 1) * cos(theta)**2)
        D = abs((b - 1) / (b + 1)) ** 0.5
        E = a * cos(theta) / (b - a * sin(theta))
        F = abs(b**2 - 1) ** 0.5
        F_v = 1/pi * ( - E * atan(D) + E * ((a**2 + (b+1)**2 - 2*b * (1 + a * sin(theta)))/ (A * B)) * atan(A * D / B) + cos(theta)/C * (atan((a * b - F**2 * sin(theta)) / (F * C)) * atan(F**2 * sin(theta)/ (F * C))))
        F_n = 1/pi * (atan(1/D) + sin(theta)/C * (atan((a * b - F**2 * sin(theta)) / (F * C) ) + atan(F**2 * sin(theta) / (F * C))) - ((a**2 + (b+1)**2 - 2 *(b+1+a*b*sin(theta)))/(A * B)) * atan(A*D/B))
        F_q = sqrt(F_v**2 + F_n**2)
        ta = exp(-7 * 10 ** -4 * (x - 0.5 * d))
        q = Ef * F_q * ta
        if round(q, 2) >= 10.5:
          result['10,5'] = round(x, 2)
        elif round(q) < 8 and round(q) >= 7:
          result['7,0'] = round(x, 2)
        elif round(q, 2) < 7 and  round(q, 2) >= 4.2:
          result['4,2'] = round(x, 2)          
        elif round(q, 2) < 2 and round(q, 2) >= 1.4:
          result['1,4'] = round(x, 2)
          return result 
          break 

        if q < 1:
          return result
          break
        x += 0.1


#Чтение и обработка данных из Excel

##Считываем данные из Excel

In [ ]:
!gdown --id 1_Oa-FbrY7uSCpUt023fqL2CwtG1PlVEa

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1_Oa-FbrY7uSCpUt023fqL2CwtG1PlVEa
To: /content/test.xlsx
100% 118k/118k [00:00<00:00, 63.4MB/s]


###Данные по веществам  df_agents

In [ ]:
df_agents = pd.read_excel('test.xlsx', index_col='Характеристики', sheet_name = 'BD_Vesh')

In [ ]:
df_agents = df_agents.drop(columns = 'размерность', axis = 1)

In [ ]:
df_agents.head(3)

,Нефть,Бензин,ДТ,Метан,ШФЛУ,БГС,ПНГ,Конденсат,Пропан,проверка
Характеристики,,,,,,,,,,
Плотность,895,750,850,0.717,680,690,0.997,680,528,NaN
Тип,ЛВЖ,ЛВЖ,ГЖ,ГФ,СУГ,ЛВЖ,ГФ,ЛВЖ,СУГ,NaN
Твсп,-25,-39,35,-200,-30,-39,-200,-100,-30,NaN


###Данные по частотам df_freq

Выбор значений частот возникновения событий, инициирующих аварии произведен на основе обобщенных статистических данных и мнения экспертов. Учитывая, что в настоящее время отсутствует нормализованный механизм по сбору статистики отказов оборудования, при использовании статистических данных из литературных источников следует оценивать степень их достоверности и понимать, что такие данные, как правило, дают лишь порядок величины. 

В данной работе при оценке риска используются сведения, рекомендованные приказом Ростехнадзора № 387 от 03.11.2022 
https://drive.google.com/file/d/1VJbsNr1Mxuxcyt2jCvMhBAPyHM8Lqheh/view?usp=share_link

In [ ]:
df_freq = pd.read_excel('test.xlsx', index_col='оборудование', sheet_name = 'BD_freq')

In [ ]:
df_freq.head(3)

,частичная,полная
оборудование,,
труба_75,1.000000e-06,5.000000e-06
труба_75_150,3.000000e-07,2.000000e-06
труба_150,1.000000e-07,5.000000e-07


###Данные по сценариям scen_lvz, scen_gj, scen_gf

In [ ]:
scen_lvz = pd.read_excel('test.xlsx', index_col = 'Сценарий', sheet_name = 'BD_Usl', skiprows = 0, nrows = 4, usecols=[1, 2, 3, 4, 5])

In [ ]:
scen_lvz

,П,М,С,Б
Сценарий,,,,
Пожар,0.2000,0.005000,0.035000,0.15000
Взрыв,0.1152,0.000398,0.008338,0.08976
Пожар-вспышка,0.0768,0.004577,0.026402,0.05984
Пролив,0.6080,0.990025,0.930260,0.70040


In [ ]:
scen_gj = pd.read_excel('test.xlsx', index_col = 'Сценарий', sheet_name = 'BD_Usl', skiprows = 6, nrows = 4, usecols=[1, 2, 3, 4, 5])

In [ ]:
scen_gj

,П,М,С,Б
Сценарий,,,,
Пожар,0.050000,0.005000,0.015000,0.040000
Взрыв,0.005795,0.000249,0.000739,0.002016
Пожар-вспышка,0.052155,0.004726,0.014036,0.038304
Пролив,0.892050,0.990025,0.970225,0.919680


In [ ]:
scen_gf = pd.read_excel('test.xlsx', index_col = 'Сценарий', sheet_name = 'BD_Usl', skiprows = 12, nrows = 4, usecols=[1, 2, 3, 4, 5])

In [ ]:
scen_gf

,П,М,С,Б
Сценарий,,,,
Факел,0.2000,0.005000,0.035000,0.15000
Взрыв,0.1152,0.000398,0.008338,0.08976
Пожар-вспышка,0.0768,0.004577,0.026402,0.05984
Пролив,0.6080,0.990025,0.930260,0.70040


###Данные по коэффициентам и ограничениям

In [ ]:
wb = ox.load_workbook('test.xlsx')

/usr/local/lib/python3.8/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
wb_info = wb['Info']

In [ ]:
#коэффициенты для расчета ущерба воде
koef_eco_water = {'Квг': wb_info['B3'].value, 
                  'Кдл': wb_info['B4'].value,
                  'Кв': wb_info['B5'].value,
                  'Кин': wb_info['B6'].value}

In [ ]:
#коэффициенты для расчета ущерба почве
koef_eco_soil = {'СХВ': wb_info['D3'].value, 
                  'Кг': wb_info['D4'].value,
                  'Кисп': wb_info['D5'].value,
                  'Тх': wb_info['D6'].value}

In [ ]:
#коэффициенты для расчета ущерба при испарении веществ
koef_eco_vapor= {'Нпл': wb_info['F3'].value, 
                  'Кот': wb_info['F4'].value,
                  'Кср': wb_info['F5'].value,
                  'Кг': wb_info['F6'].value}

In [ ]:
#коэффициенты для расчета ущерба при сгорании веществ
koef_eco_fire= {'Тх': wb_info['H3'].value, 
                  'Кср': wb_info['H4'].value,
                  'Кг': wb_info['H5'].value}

In [ ]:
#ограничения: максимальная смена, скорость движения аварийной бригады
bd_limit = {'max_smena': wb_info['B9'].value, 
                'speed_inspector': round(wb_info['B12'].value / 3.6,2)}

###Данные по оборудованию (основной датафрейм)

In [ ]:
df = pd.read_excel('test.xlsx', index_col='№', sheet_name = 'Oborud')

In [ ]:
df.head(3)

,наименование оборудования,Тип оборудования,Вещество,Тип содержимого,"Количество опасного вещества, т",Температура,давление,"Интенсивность прокачки, м3/сут",время перекрытия,"Протяженность, м",диаметр,толщина стенки,Коэффициент подстил поверхности,Класс загроможденности пространства,Блок,№ в блоке,частота обхода (дней),пересечения - водотоки,пересечения - ЖД,"пересечения - АД, категория дорог"
№,,,,,,,,,,,,,,,,,,,,
1,"нефтепровод (1000х12, 10000 м)",труба,Нефть,ЛВЖ,NaN,-20..20,4,26.0,300,10000,1000,12,5,4,Система промысловых трубопроводов,1,3,NaN,NaN,NaN
2,"нефтепровод (1000х12, 5000 м)",труба,Нефть,ЛВЖ,NaN,-20..20,4,142.0,300,5000,1000,12,5,4,Система промысловых трубопроводов,2,3,1.0,NaN,NaN
3,"нефтепровод (1000х12, 1000 м)",труба,Нефть,ЛВЖ,NaN,-20..20,4,142.0,300,1000,1000,12,5,4,Система промысловых трубопроводов,3,3,1.0,NaN,NaN


##Объем и масса по каждому участку трубопровода

In [ ]:
df['объем'] = round(volume_pipeline(df['Протяженность, м'], df['диаметр'], df['толщина стенки']), 2)

In [ ]:
df = pd.merge(df, df_agents.loc['Плотность'], left_on='Вещество', right_index=True)

In [ ]:
df['Плотность'] = df['Плотность']/1000

In [ ]:
df['Количество опасного вещества, т'] = df['Количество опасного вещества, т'].fillna(0)

In [ ]:
df['масса_в_участке'] = np.where(df['Количество опасного вещества, т'] > 0, df['Количество опасного вещества, т'], df['объем'] * df['Плотность'])

##Масса вещества участвующего в аварии и площади пролива

###Площади отверстий

In [ ]:
df['площадь_гильотины'] = hole_area(df['диаметр'], df['толщина стенки'], type='гильотина')

In [ ]:
df['площадь_малой'] = hole_area(df['диаметр'], df['толщина стенки'], type='малая')

In [ ]:
df['площадь_свища'] = hole_area(df['диаметр'], df['толщина стенки'], type='свищ')

###Напорный и безнапорный режим для гильотины, малой трещины и свища

In [ ]:
df['напор гильотины'] = df.apply(lambda x: flow_pipe(x['давление'], x['площадь_гильотины'], x['Плотность']), axis=1)

In [ ]:
df['напор малой'] = df.apply(lambda x: flow_pipe(x['давление'], x['площадь_малой'], x['Плотность']), axis=1)

In [ ]:
df['напор свища'] = df.apply(lambda x: flow_pipe(x['давление'], x['площадь_свища'], x['Плотность']), axis=1)

In [ ]:
df['безнапор гильотины'] = df.apply(lambda x: flow_pipe(1.2, x['площадь_гильотины'], x['Плотность']), axis=1)

In [ ]:
df['безнапор малой'] = df.apply(lambda x: flow_pipe(1.2, x['площадь_малой'], x['Плотность']), axis=1)

In [ ]:
df['безнапор свища'] = df.apply(lambda x: flow_pipe(1.2, x['площадь_свища'], x['Плотность']), axis=1)

###Массы V1, V2, V3 для трубопровода 

In [ ]:
df['V1_свищ'] = df.loc[:, 'напор свища'] * df.loc[:, 'частота обхода (дней)'] * 86400 / 1000

In [ ]:
df['V1_малая'] = df.loc[:, 'напор малой'] * df.loc[:, 'частота обхода (дней)'] * 86400 / 1000

In [ ]:
df['V1_гильотина'] = df.loc[:, 'напор гильотины'] * df.loc[:, 'время перекрытия'] / 1000

In [ ]:
df['V2_свищ'] = df.loc[:, 'безнапор свища'] * df.loc[:, 'время перекрытия'] / 1000

In [ ]:
df['V2_малая'] = df.loc[:, 'безнапор малой'] * df.loc[:, 'время перекрытия'] / 1000

In [ ]:
df['V2_гильотина'] = df.loc[:, 'безнапор гильотины'] * df.loc[:, 'время перекрытия'] / 1000

In [ ]:
df['V3_свищ'] = df.loc[:, 'безнапор свища'] * (df.loc[:, 'Протяженность, м'] / bd_limit['speed_inspector']) / 1000

In [ ]:
df['V3_малая'] = df.loc[:, 'безнапор малой'] * (df.loc[:, 'Протяженность, м'] / bd_limit['speed_inspector']) / 1000

In [ ]:
df['V3_гильотина'] = df.loc[:, 'безнапор гильотины'] * (df.loc[:, 'Протяженность, м'] / bd_limit['speed_inspector']) / 1000

In [ ]:
df['V3_свищ_АСФ'] = df.loc[:, 'безнапор свища'] * 3600 / 1000

In [ ]:
df['V3_малая_АСФ'] = df.loc[:, 'безнапор малой'] * 3600 / 1000

In [ ]:
df['V3_гильотина_АСФ'] = df.loc[:, 'безнапор гильотины'] * 3600 / 1000

In [ ]:
df['масса в аварии свищ'] = df.loc[:, 'V1_свищ'] + np.where(df.loc[:, 'масса_в_участке'] < df.loc[:, 'V2_свищ'] + df.loc[:, 'V3_свищ'], 
                                                            df.loc[:, 'масса_в_участке'], 
                                                            df.loc[:, 'V2_свищ'] + df.loc[:, 'V3_свищ'])

In [ ]:
df['масса в аварии малая'] = df.loc[:, 'V1_малая'] + np.where(df.loc[:, 'масса_в_участке'] < df.loc[:, 'V2_малая'] + df.loc[:, 'V3_малая'], 
                                                            df.loc[:, 'масса_в_участке'], 
                                                            df.loc[:, 'V2_малая'] + df.loc[:, 'V3_малая'])

In [ ]:
df['масса в аварии гильотина'] = df.loc[:, 'V1_гильотина'] + np.where(df.loc[:, 'масса_в_участке'] < df.loc[:, 'V2_гильотина'] + df.loc[:, 'V3_гильотина'], 
                                                            df.loc[:, 'масса_в_участке'], 
                                                            df.loc[:, 'V2_гильотина'] + df.loc[:, 'V3_гильотина'])

###Тестово! Расчет массы по ПП РФ
Постановление Правительства РФ от 31 декабря 2020 г. N 2451

В случае аварии, связанной с **разгерметизацией (порывом)** напорного нефтепровода, количество опасного вещества, участвующего в аварии, принималось равным сумме 25 процентов максимального объема прокачки в течение 6 часов и объема нефти между запорными задвижками на порванном участке трубопровода.

Для оценки последствий таких типичных аварий для напорного нефтепровода, как **прокол**, представляющих опасность в первую очередь с точки зрения нанесения экологического ущерба, объем разлива нефти определялся в соответствии с Постановлением Правительства РФ от 31.12.2020 г. № 2451 «Об утверждении Правил организации мероприятий по предупреждению и ликвидации разливов нефти и нефтепродуктов на территории Российской Федерации, за исключением внутренних морских вод Российской Федерации и территориального моря Российской Федерации, а также о признании утратившими силу некоторых актов Правительства Российской Федерации». При этом объем разлива составляет 2 % максимального объема прокачки за время между последовательным осмотром (мониторингом), установленное графиками обхода.


Гильотина

In [ ]:
df['гильотина_ПП'] = df.apply(lambda x: massa_pp_gap(x['Интенсивность прокачки, м3/сут'], x['Плотность'], x['масса_в_участке']), axis=1)

Свищ

In [ ]:
df['свищ_ПП'] = df.apply(lambda x: massa_pp_puncture(x['Интенсивность прокачки, м3/сут'], x['Плотность'], x['частота обхода (дней)']), axis=1)

###Площади пролива

In [ ]:
df['площадь полная'] = Strait_area(df['масса в аварии гильотина'], df['Плотность'], df['Коэффициент подстил поверхности'])

In [ ]:
df['площадь частичная'] = Strait_area(df['масса в аварии свищ'], df['Плотность'], df['Коэффициент подстил поверхности'])

In [ ]:
df['облако'] = Evaporation_404() * 3600 * df['площадь полная']

In [ ]:
df['твс'] = df['облако'] * 0.1

In [ ]:
df[['площадь полная', 'площадь частичная', 'облако', 'твс']].head()

,площадь полная,площадь частичная,облако,твс
№,,,,
1,48657.877237,7316.262535,20357.106659,2035.710666
2,29954.127237,7289.523312,12531.976273,1253.197627
3,14991.077237,7268.131933,6271.851046,627.185105
4,13120.727237,7265.458011,5489.348467,548.934847
5,11624.427237,7263.318873,4863.338036,486.333804


##С-2 Сценарий пожар-пролива

In [ ]:
 df[['пожар_пролива_10,5', 'пожар_пролива_7,0','пожар_пролива_4,2', 'пожар_пролива_1,4']] = df['площадь полная'].apply(lambda x: pd.Series(StraitFire_404(x)))

In [ ]:
df[['пожар_пролива_10,5', 'пожар_пролива_7,0','пожар_пролива_4,2', 'пожар_пролива_1,4']].head(3)

,"пожар_пролива_10,5","пожар_пролива_7,0","пожар_пролива_4,2","пожар_пролива_1,4"
№,,,,
1,319.9,386.2,457.1,605.1
2,259.4,315.2,375.1,501.4
3,191.9,235.2,281.9,381.6


##С-3 Сценарий пожар вспышка

In [ ]:
df['вспышка'] = fire_flash_404(df['облако'])

##С-4 Сценарий Взрыв ТВС

In [ ]:
 df[['твс_100', 'твс_53','твс_28', 'твс_12', 'твс_5', 'твс_3']] = df['твс'].apply(lambda x: pd.Series(TVS_404(x)))

In [ ]:
 df[['твс_100', 'твс_53','твс_28', 'твс_12', 'твс_5', 'твс_3']].head(3)

,твс_100,твс_53,твс_28,твс_12,твс_5,твс_3
№,,,,,,
1,-,-,-,101.5,274.6,274.7
2,-,-,-,70.0,196.2,196.3
3,-,-,-,39.5,120.8,120.9


##Определение вероятности реализации событий

Расчитываем вероятность возникновения иницирующих событий с учетом длины трубопроводов

In [ ]:
df[['полная разгерметизация', 'частичная (прокол)']] = df['диаметр'].apply(lambda x: pd.Series(frequencies_pipline(x))) * df['Протяженность, м'].values.reshape(-1, 1)

Расчитываем вероятность возникновения различных сценариев

In [ ]:
scen_lvz.loc["Пожар", 'П']

0.2

In [ ]:
df['С-1А(п)'] = df['полная разгерметизация'] * scen_lvz.loc['Пролив', 'П']
df['С-1А(ч)'] = df['частичная (прокол)'] * scen_lvz.loc['Пролив', 'С']
df['С-2'] = df['полная разгерметизация'] * scen_lvz.loc['Пожар', 'П']
df['С-3'] = df['полная разгерметизация'] * scen_lvz.loc['Пожар-вспышка', 'П']
df['С-4'] = df['полная разгерметизация'] * scen_lvz.loc['Взрыв', 'П']

##Ущерб

In [ ]:
df.columns

Index(['наименование оборудования', 'Тип оборудования', 'Вещество',
       'Тип содержимого', 'Количество опасного вещества, т', 'Температура',
       'давление', 'Интенсивность прокачки, м3/сут', 'время перекрытия',
       'Протяженность, м', 'диаметр', 'толщина стенки',
       'Коэффициент подстил поверхности',
       'Класс загроможденности пространства', 'Блок', '№ в блоке',
       'частота обхода (дней)', 'пересечения - водотоки', 'пересечения - ЖД',
       'пересечения - АД, категория дорог', 'объем', 'Плотность',
       'масса_в_участке', 'площадь_гильотины', 'площадь_малой',
       'площадь_свища', 'напор гильотины', 'напор малой', 'напор свища',
       'безнапор гильотины', 'безнапор малой', 'безнапор свища', 'V1_свищ',
       'V1_малая', 'V1_гильотина', 'V2_свищ', 'V2_малая', 'V2_гильотина',
       'V3_свищ', 'V3_малая', 'V3_гильотина', 'V3_свищ_АСФ', 'V3_малая_АСФ',
       'V3_гильотина_АСФ', 'масса в аварии свищ', 'масса в аварии малая',
       'масса в аварии гильотина', '

#Заполнение данных в Excel

##Количество вещества в оборудовании

In [ ]:
wb_rasp = wb['Rasp']

In [ ]:
for i, r in df.iterrows():
  wb_rasp.cell(row = i + 3, column = 1, value = df['Блок'][i])
  wb_rasp.cell(row = i + 3, column = 2, value = f'''{df['наименование оборудования'][i]}, {df['Вещество'][i]}''')
  wb_rasp.cell(row = i + 3, column = 3, value = f'''{df['Протяженность, м'][i]}, м''')
  wb_rasp.cell(row = i + 3, column = 4, value = f'''{str(round(df['масса_в_участке'][i], 2)).replace('.', ',')}''')
  wb_rasp.cell(row = i + 3, column = 4).number_format = '0,00'
  wb_rasp.cell(row = i + 3, column = 6, value = f'''Жидкость''')
  wb_rasp.cell(row = i + 3, column = 7, value = f'''{df['давление'][i]}''')
  wb_rasp.cell(row = i + 3, column = 8, value = f'''{df['Температура'][i]}''') 
wb_rasp.cell(row = 4, column = 5, value = str(df.groupby("Блок")["масса_в_участке"].sum()[0]).replace('.', ','))
wb_rasp.merge_cells(start_row=4, start_column=5, end_row=len(df)+3, end_column=5)
#итог
wb_rasp.cell(row = len(df)+4, column = 1, value = 'Итого опасного вещества - Нефть,т')
wb_rasp.cell(row = len(df)+4, column = 5, value = str(df.groupby("Блок")["масса_в_участке"].sum()[0]).replace('.', ','))
wb_rasp.merge_cells(start_row=len(df)+4, start_column=1, end_row=len(df)+4, end_column=4)
wb_rasp.merge_cells(start_row=len(df)+4, start_column=5, end_row=len(df)+4, end_column=8)
wb_rasp.cell(row = len(df)+5, column = 1, value = 'из них - в сосудах (аппаратах), т')
wb_rasp.cell(row = len(df)+5, column = 5, value = '0')
wb_rasp.merge_cells(start_row=len(df)+5, start_column=1, end_row=len(df)+5, end_column=4)
wb_rasp.merge_cells(start_row=len(df)+5, start_column=5, end_row=len(df)+5, end_column=8)
wb_rasp.cell(row = len(df)+6, column = 1, value = 'в трубопроводах, т')
wb_rasp.cell(row = len(df)+6, column = 5, value = str(df.groupby("Блок")["масса_в_участке"].sum()[0]).replace('.', ','))
wb_rasp.merge_cells(start_row=len(df)+6, start_column=1, end_row=len(df)+6, end_column=4)
wb_rasp.merge_cells(start_row=len(df)+6, start_column=5, end_row=len(df)+6, end_column=8)

##Массы по сценариям

In [ ]:
wb_mass = wb['Masses']

In [ ]:
j = 4
for i, r in df.iterrows():
  wb_mass.cell(row = j, column = 1, value = 'С-1А')
  wb_mass.cell(row = j + 1, column = 1, value = 'С-2')
  wb_mass.cell(row = j + 2, column = 1, value = 'С-3')
  wb_mass.cell(row = j + 3, column = 1, value = 'С-4А')
  #column 2
  wb_mass.cell(row = j, column = 2, value = f'''{df['наименование оборудования'][i]}''')
  wb_mass.merge_cells(start_row=j, start_column=2, end_row=j+3, end_column=2)
  #column 3
  wb_mass.cell(row = j, column = 3, value = 'Пролив нефти на открытой площадке')
  wb_mass.cell(row = j + 1, column = 3, value = 'Пожар пролива на открытой площадке')
  wb_mass.cell(row = j + 2, column = 3, value = 'Пожар-вспышка на открытой площадке')
  wb_mass.cell(row = j + 3, column = 3, value = 'Взрыв ГПВС на открытой площадке') 
  #column 4
  wb_mass.cell(row = j, column = 4, value = 'Экологическое загрязнение')
  wb_mass.cell(row = j + 1, column = 4, value = 'Открытое пламя, тепловое излучение')
  wb_mass.cell(row = j + 2, column = 4, value = 'Высокотемпературные продукты сгорания')
  wb_mass.cell(row = j + 3, column = 4, value = 'Ударная волна, осколки')
  #column 5
  wb_mass.cell(row = j, column = 5, value = f'''{round(df['масса в аварии гильотина'][i], 2)} / {round(df['масса в аварии свищ'][i],2)}''')
  wb_mass.merge_cells(start_row=j, start_column=5, end_row=j+3, end_column=5)
  #column 6
  wb_mass.cell(row = j, column = 6, value = f'''{round(df['масса в аварии гильотина'][i], 2)} / {round(df['масса в аварии свищ'][i],2)}''')
  wb_mass.cell(row = j + 1, column = 6, value = round(df['масса в аварии гильотина'][i], 2))
  wb_mass.cell(row = j + 2, column = 6, value = round(df['облако'][i], 2))
  wb_mass.cell(row = j + 3, column = 6, value = round(df['твс'][i], 2))  
  j = j + 4
 

##Напор

In [ ]:
wb_napor = wb['Napor']

In [ ]:
for i, r in df.iterrows():
  wb_napor.cell(row = i + 1, column = 1, value = df.loc[i, 'наименование оборудования'])
  wb_napor.cell(row = i + 1, column = 2, value = df.loc[i, 'V1_свищ'])
  wb_napor.cell(row = i + 1, column = 3, value = df.loc[i, 'V2_свищ'])
  wb_napor.cell(row = i + 1, column = 4, value = df.loc[i, 'V3_свищ'])
  wb_napor.cell(row = i + 1, column = 5, value = df.loc[i, 'V1_малая'])
  wb_napor.cell(row = i + 1, column = 6, value = df.loc[i, 'V2_малая'])
  wb_napor.cell(row = i + 1, column = 7, value = df.loc[i, 'V3_малая'])
  wb_napor.cell(row = i + 1, column = 8, value = df.loc[i, 'V1_гильотина'])
  wb_napor.cell(row = i + 1, column = 9, value = df.loc[i, 'V2_гильотина'])
  wb_napor.cell(row = i + 1, column = 10, value = df.loc[i, 'V3_гильотина'])
  wb_napor.cell(row = i + 1, column = 11, value = df.loc[i, 'V3_свищ_АСФ'])
  wb_napor.cell(row = i + 1, column = 12, value = df.loc[i, 'V3_малая_АСФ'])
  wb_napor.cell(row = i + 1, column = 13, value = df.loc[i, 'V3_гильотина_АСФ'])
  wb_napor.cell(row = i + 1, column = 14, value = df.loc[i, 'свищ_ПП'])
  wb_napor.cell(row = i + 1, column = 15, value = df.loc[i, 'гильотина_ПП'])

##Сценарий С-1

In [ ]:
wb_c1 = wb['C1']

In [ ]:
df.columns

Index(['наименование оборудования', 'Тип оборудования', 'Вещество',
       'Тип содержимого', 'Количество опасного вещества, т', 'Температура',
       'давление', 'Интенсивность прокачки, м3/сут', 'время перекрытия',
       'Протяженность, м', 'диаметр', 'толщина стенки',
       'Коэффициент подстил поверхности',
       'Класс загроможденности пространства', 'Блок', '№ в блоке',
       'частота обхода (дней)', 'пересечения - водотоки', 'пересечения - ЖД',
       'пересечения - АД, категория дорог', 'объем', 'Плотность',
       'масса_в_участке', 'площадь_гильотины', 'площадь_малой',
       'площадь_свища', 'напор гильотины', 'напор малой', 'напор свища',
       'безнапор гильотины', 'безнапор малой', 'безнапор свища', 'V1_свищ',
       'V1_малая', 'V1_гильотина', 'V2_свищ', 'V2_малая', 'V2_гильотина',
       'V3_свищ', 'V3_малая', 'V3_гильотина', 'V3_свищ_АСФ', 'V3_малая_АСФ',
       'V3_гильотина_АСФ', 'масса в аварии свищ', 'масса в аварии малая',
       'масса в аварии гильотина', '

In [ ]:
j = 4
for i, r in df.iterrows():
  #column 1
  wb_c1.cell(row = j, column = 1, value = 'С-1А(п)')
  wb_c1.merge_cells(start_row=j, start_column=1, end_row=j+1, end_column=1)
  wb_c1.cell(row = j + 2, column = 1, value = 'С-1А(ч)')
  wb_c1.merge_cells(start_row=j+2, start_column=1, end_row=j+3, end_column=1)
  #column 2
  wb_c1.cell(row = j, column = 2, value = f'''{df['наименование оборудования'][i]}''')
  wb_c1.merge_cells(start_row=j, start_column=2, end_row=j+3, end_column=2) 
  #column 3
  wb_c1.cell(row = j, column = 3, value = 'Площадь пролива, м2')
  wb_c1.cell(row = j + 1, column = 3, value = 'Радиус пролива, м')
  wb_c1.cell(row = j + 2, column = 3, value = 'Площадь пролива, м2')
  wb_c1.cell(row = j + 3, column = 3, value = 'Радиус пролива, м')
  #column 4
  wb_c1.cell(row = j, column = 4, value = round(df['площадь полная'][i], 2))
  wb_c1.cell(row = j + 1, column = 4, value = round(df['площадь полная'][i] / pi, 2))
  wb_c1.cell(row = j + 2, column = 4, value = round(df['площадь частичная'][i], 2))
  wb_c1.cell(row = j + 3, column = 4, value = round(df['площадь частичная'][i] / pi, 2))   
  j += 4


##Сценарий С-2

In [ ]:
wb_c2 = wb['C2']

In [ ]:
j = 4
for i, r in df.iterrows():
  #column 1
  wb_c2.cell(row = j, column = 1, value = df['наименование оборудования'][i])
  wb_c2.merge_cells(start_row=j, start_column=1, end_row=j+4, end_column=1)
  #column 2
  wb_c2.cell(row = j, column = 2, value = 'Площадь пожара, м2')
  wb_c2.cell(row = j+1, column = 2, value = 'Радиусы зон поражения тепловым излучением, м:')
  wb_c2.cell(row = j+2, column = 2, value = '– непереносимая боль через 3-5 с (10,5 кВт/м2)')
  wb_c2.cell(row = j+3, column = 2, value = '– непереносимая боль через 20-30 с (7,0 кВт/м2)')
  wb_c2.cell(row = j+4, column = 2, value = '– зона отсутствия негативных последствий (1,4 кВт/м2)')  
  #column 3
  wb_c2.cell(row = j, column = 3, value = round(df['площадь полная'][i], 2))
  wb_c2.cell(row = j + 2, column = 3, value = df['пожар_пролива_10,5'][i])
  wb_c2.cell(row = j + 3, column = 3, value = df['пожар_пролива_7,0'][i])
  wb_c2.cell(row = j + 4, column = 3, value = df['пожар_пролива_1,4'][i])   
  j += 5

##Сценарий С-3

In [ ]:
wb_c3 = wb['C3']

In [ ]:
j = 4
for i, r in df.iterrows():
  #column 1
  wb_c3.cell(row = j, column = 1, value = df['наименование оборудования'][i])
  #column 2
  wb_c3.cell(row = j, column = 2, value = 'Радиус зоны воздействия высокотемпературных продуктов сгорания, м') 
  #column 3
  wb_c3.cell(row = j, column = 3, value = round(df['вспышка'][i], 2))  
  j += 1

## Сценарий С-4

In [ ]:
wb_c4 = wb['C4']

In [ ]:
j = 4
for i, r in df.iterrows():
  #column 1
  wb_c4.cell(row = j, column = 1, value = df['наименование оборудования'][i])
  #column 2
  wb_c4.cell(row = j, column = 2, value = df['твс_100'][i]) 
  #column 3
  wb_c4.cell(row = j, column = 3, value = df['твс_53'][i]) 
  #column 4
  wb_c4.cell(row = j, column = 4, value = df['твс_28'][i]) 
  #column 5
  wb_c4.cell(row = j, column = 5, value = df['твс_12'][i])
  #column 6
  wb_c4.cell(row = j, column = 6, value = df['твс_5'][i])
  #column 7
  wb_c4.cell(row = j, column = 7, value = df['твс_3'][i])
  j += 1

##Вероятность реализации

In [ ]:
chast_tr = wb['Chast_tr']

In [ ]:
j = 3
for i, r in df.iterrows():
  #column 1
  chast_tr.cell(row = j, column = 1, value = 'С-1А(ч)')
  chast_tr.cell(row = j+1, column = 1, value = 'С-1А(п)')
  chast_tr.cell(row = j+2, column = 1, value = 'С-2')
  chast_tr.cell(row = j+3, column = 1, value = 'С-3')
  chast_tr.cell(row = j+4, column = 1, value = 'С-4')
  #column 2
  chast_tr.cell(row = j, column = 2, value = df['наименование оборудования'][i])
  chast_tr .merge_cells(start_row=j, start_column=2, end_row=j+4, end_column=2)
  #column 3  
  chast_tr.cell(row = j, column = 3, value = df['С-1А(ч)'][i])
  chast_tr.cell(row = j+1, column = 3, value = df['С-1А(п)'][i])
  chast_tr.cell(row = j+2, column = 3, value = df['С-2'][i])
  chast_tr.cell(row = j+3, column = 3, value = df['С-3'][i])
  chast_tr.cell(row = j+4, column = 3, value = df['С-4'][i])
  j += 5


##Сохранение

In [ ]:
wb.save('/content/drive/MyDrive/Work, Study, Life/Analyst/Pipeline_method/result.xlsx')

https://docs.google.com/spreadsheets/d/1w_QSPteFTJeMEs52QoI4aqBZuaj-34MG/edit?usp=share_link&ouid=106885717380627810664&rtpof=true&sd=true